# 데이터 분할을 위한 하위 폴더 생성

In [ ]:
import os
import shutil



original_datset_dir = '../input/plant-leaf-diseases-without-aug/Plant_leave_diseases_dataset_with_augmentation' # 원본 데이터셋의 경로를 지정.
classes_list = os.listdir(original_datset_dir) # 하위에 있는 모든 폴더의 목록을 클래스 이름으로 가져온다.

base_dir = './splitted'

os.mkdir(base_dir) # 용도별로 나눌 폴더를 생성.

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val')
os.mkdir(val_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(val_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))
    


# 데이터 분할과 클래스별 데이터 수 확인

In [ ]:
import math

for cls in classes_list:
    path = os.path.join(original_datset_dir, cls)
    fnames = os.listdir(path) # 각 클래스마다 존재하는 모든 이미지 파일의 목록을 fname에 저장한다.
    
    train_size = math.floor(len(fnames) * 0.6)
    val_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)    
    
    train_fnames = fnames[:train_size]
    print('Train size(',cls,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst) # src의 경로에 해당하는 파일을 dst의 경로에 저장한다. 
        
    val_fnames = fnames[train_size : (val_size + train_size)]
    #print('Validation size(',cls,'): ', len(val_fnames))
    for fname in val_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(val_dir, cls), fname)
        shutil.copyfile(src, dst) # src의 경로에 해당하는 파일을 dst의 경로에 저장한다. 
        

    test_fnames = fnames[(val_size + train_size) : (val_size + train_size + test_size)]
    #print('Test size(',cls,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst) # src의 경로에 해당하는 파일을 dst의 경로에 저장한다. 
        

# 베이스라인 모델 학습을 위한 준비

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torchvision
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)
batch_size = 64
epoch = 30

# Weighted Random Sampling을 위한 함수 정의

def make_weights_for_balanced_classes(img, nclasses):

    labels = []
    for i in range(len(img)):
        labels.append(img[i][1])

    label_array = np.array(labels)
    total = len(labels)

    count_list = []
    for cls in range(nclasses):
        count = len(np.where(label_array == cls)[0])
        count_list.append(total/count)

    weights = []
    for label in label_array:
        weights.append(count_list[label])

    return weights




transf = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

train_dataset = ImageFolder(root = './splitted/train', transform = transf)
val_dataset = ImageFolder(root = './splitted/val', transform = transf)

weights_train = make_weights_for_balanced_classes(train_dataset.imgs, len(train_dataset.classes)) # 가중치 계산
weights_train = torch.DoubleTensor(weights_train) # 텐서 변환
sampler_train = torch.utils.data.sampler.WeightedRandomSampler(weights_train, len(weights_train)) # 샘플링 방법 정의

weights_val = make_weights_for_balanced_classes(val_dataset.imgs, len(val_dataset.classes)) # 가중치 계산
weights_val = torch.DoubleTensor(weights_val) # 텐서 변환
sampler_val = torch.utils.data.sampler.WeightedRandomSampler(weights_val, len(weights_val)) # 샘플링 방법 정의

train_loader = DataLoader(train_dataset, batch_size = batch_size, sampler = sampler_train) # 데이터 로더 정의
val_loader = DataLoader(val_dataset, batch_size = batch_size, sampler = sampler_val) # 데이터 로더 정의

#train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # 데이터 로더 정의
#val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

print(len(train_dataset.classes))



# 베이스라인 모델 설계

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, padding = 1)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 39)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = F.dropout(x, p = 0.25, training = self.training)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = F.dropout(x, p = 0.25, training = self.training)
        
        x = self.conv3(x)
        #x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = F.dropout(x, p = 0.25, training = self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = self.relu(x)
        x = F.dropout(x, p = 0.5, training = self.training) 
        x = self.fc2(x) 
        
        return F.log_softmax(x, dim = 1)
    
model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        

# 모델 학습과 평가를 위한 함수

In [ ]:
def train(model, train_loader, optimizer):
    model.train() # 입력 받는 모델을 학습 모드로 설정.
    
    for (x, y) in train_loader:
        x = x.to(device)
        y = y.to(device)
   # for batch_idx, (x, y) in enumerate(train_loader):  
    #    x, y = x.to(device), y.to(device)
            
        optimizer.zero_grad() # 이전 batch의 gradient값이 옵티마이저에 저장되어 있으므로 초기화 시켜준다.
        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)
        loss.backward() # 계산한 loss값을 바탕으로 역전파를 통해 계산한 gradient 값을 각 parameter에 할당한다.
        optimizer.step() # 각 parameter에 할당된 gradient 값을 이용해 모델의 parameter를 업데이트 한다.
        
def evaluate(model, test_loader):
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    
    model.eval() # 입력 받는 모델을 평가 모드로 설정. 
    with torch.no_grad(): # 해당 메서드를 이용하여 해당 부분을 실행하는 동안 모델의 Parameter 업데이트를 중단한다. 
        for (x, y) in test_loader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            
            epoch_loss += F.cross_entropy(y_pred, y, reduction = 'sum').item()
            pred = y_pred.max(1, keepdim = True)[1] # 모델에 입력된 test 데이터가 33개의 클래스에 속할 각각의 확률값을 반환 -> 이 중 가장 높은 값을 가진 인덱스를 예측값으로 pred에 저장.
            correct += pred.eq(y.view_as(pred)).sum().item() # y.view_as(pred)를 통해 y Tensor의 구조를 pred Tensor의 모양대로 재정렬 -> 일치하는 갯수만큼 반환
            
    epoch_loss /= len(test_loader.dataset)
    epoch_acc = 100. * correct / len(test_loader.dataset)
    return epoch_loss, epoch_acc
        
        
        
        


# 모델 학습 실행하기

In [ ]:
import time
import copy

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    #print('1')
    best_model_weights = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
       # print('2')
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_weights = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('---------- epoch {} ----------'.format(epoch))
        print('Train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('Validation Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    model.load_state_dict(best_model_weights)
    return model

base_model = train_baseline(model, train_loader, val_loader, optimizer, epoch)
torch.save(base_model, 'base_model.pt')

# Transfer Learning을 위한 준비

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted'
image_datasets = {x: ImageFolder(root = os.path.join(data_dir, x), transform = data_transforms[x])
                     for x in {'train', 'val'}}
dataloaders = {x: DataLoader(image_datasets[x], batch_size = batch_size, shuffle = True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Pre-Trained Model 불러오기

In [ ]:
from torchvision import models
from torch.optim import lr_scheduler


resnet = models.resnet50(pretrained = True)
n_filters = resnet.fc.in_features # 불러온 ResNet50의 마지막 Layer의 입력 채널의 수.
resnet.fc = nn.Linear(n_filters, 39)# 불러온 모델의 마지막 FC 레이어를 새로운 레이어로 교체
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad, resnet.parameters()), lr = 1e-3) # requires_grad = True로 설정된 레이어의 parameter만 업데이트.


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)


# Pre-Trained 모델의 일부 Layer Freeze 시키기

In [ ]:
count = 0
for child in resnet.children(): # .children() -> 모델의 자식 모듈을 iterable한 객체로 반환, 즉 resnet 모델의 모든 레이어 정보를 담고 있다.
    count += 1
    if count < 6:
        for param in child.parameters(): # ResNet50에 존재하는 10개의 레이어 중에서 1-5번 레이어의 parameter를 freeze.
            param.rquires_grad = False

# Transfer Learning 모델 학습과 검증을 위한 함수

In [ ]:
def train_resnet(model, optimizer, criterion, scheduler, num_epochs = 15):
    best_acc = 0.0
    best_model_weights = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        print('---------- epoch {} ----------'.format(epoch))
        since = time.time()
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for (x, y) in dataloaders[phase]:
                x = x.to(device)
                y = y.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # set_grad_enabled() 메서드를 이용하여 PHASE가 TRAIN일 경우에만 모델의 Gradient가 업데이트 되도록 설정.
                    y_pred = model(x)
                    _, predicted = torch.max(y_pred, 1) 
                    loss = criterion(y_pred, y)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                epoch_loss += loss.item() * x.size(0)
                epoch_corrects += torch.sum(predicted == y.data)
            
            if phase == 'train':
                scheduler.step() # 설정된 7번의 Epoch마다 학습률을 다르게 조정하는 것을 반영. 
                lr = [x['lr'] for x in optimizer_ft.param_groups] # 학습률이 조정되고 있는것을 확인하기 위해 각 Epoch의 lr parameter를 불러온다.
                print('learning_rate: ', lr)
                
                
            epoch_loss /= dataset_sizes[phase]
            epoch_acc = epoch_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    print('Best validation Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_weights)
    
    return model
                    
                    
                    


    

# 모델 학습 실행하기

In [ ]:
model_resnet50 = train_resnet(resnet, optimizer_ft, criterion, exp_lr_scheduler, num_epochs = 15)
torch.save(model_resnet50, 'resnet50.pt') 


# 베이스라인 모델 평가를 위한 전처리 과정

In [51]:
test_base = ImageFolder(root = './splitted/test', transform = transf)
test_loader_base = DataLoader(test_base, batch_size = batch_size, shuffle = True)
                                

# Transfer Learning 모델 평가를 위한 전처리 과정

In [52]:
transf_resNet = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
])

test_resNet = ImageFolder(root = './splitted/test', transform = transf_resNet)
test_loader_resNet = DataLoader(test_resNet, batch_size = batch_size, shuffle = True)

# 베이스라인 모델 성능 평가

In [53]:
baseline = torch.load('base_model.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('Baseline test Acc: ', test_accuracy)

# Transfer Learning 모델 성능 평가

In [54]:
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test Acc: ', test_accuracy)